# Setup
This IPython notebook will walk through the steps of characterizing iModulons through the semi-automated tools in PyModulon. You will need:

* M and A matrices
* Expression data (e.g. `log_tpm_norm.csv`)
* Gene table and KEGG/GO annotations (Generated in `gene_annotation.ipynb`)
* Sample table, with a column for `project` and `condition`
* TRN file

Optional:
* iModulon table (if you already have some characterized iModulons)

In [1]:
from pymodulon.core import IcaData
from pymodulon.plotting import *
from os import path
import pandas as pd
import re
from Bio.KEGG import REST
from tqdm.notebook import tqdm

In [2]:
project_dir = path.join('..','Species','Synechocystis_sp_PCC_6803')

# Enter the location of your data here
data_dir = path.join(project_dir,'computed_data')

# GO and KEGG annotations are in the 'external' folder
external_data = path.join(project_dir,'external')

## Check your sample table (i.e. metadata file)
Your metadata file will probably have a lot of columns, most of which you may not care about. Feel free to save a secondary copy of your metadata file with only columns that seem relevant to you. The two most important columns are:
1. `project`
2. `condition`

Make sure that these columns exist in your metadata file

In [3]:
df_metadata = pd.read_csv(path.join(data_dir,'final_metadata.tsv'),index_col=0,sep='\t')
df_metadata[['project','condition']].head()

,project,condition
SRX12668170,thermal,Control_30_0
SRX12668171,thermal,Control_30_1
SRX12668172,thermal,Control_30_2
SRX12668179,thermal,Control_30_9
SRX12668183,thermal,Control_30_1


In [97]:
print(df_metadata.project.notnull().all())
print(df_metadata.condition.notnull().all())

True
True


## Check your TRN

Each row of the TRN file represents a regulatory interaction.  
**Your TRN file must have the following columns:**
1. `regulator` - Name of regulator (`/` or `+` characters will be converted to `;`)
1. `gene_id` - Locus tag of gene being regulated

The following columns are optional, but are helpful to have:
1. `regulator_id` - Locus tag of regulator
1. `gene_name` - Name of gene (can automatically update this using `name2num`)
1. `direction` - Direction of regulation ('+' for activation, '-' for repression, '?' or NaN for unknown)
1. `evidence` - Evidence of regulation (e.g. ChIP-exo, qRT-PCR, SELEX, Motif search)
1. `PMID` - Reference for regulation

You may add any other columns that could help you. TRNs may be saved as either CSV or TSV files. See below for an example:

In [93]:
df_trn = pd.read_csv(path.join(external_data,'TRN.csv'))
df_trn.head()

FileNotFoundError: [Errno 2] No such file or directory: '../Species/Synechocystis_sp_PCC_6803/external/TRN.csv'

The `regulator` and `gene_id` must be filled in for each row

In [ ]:
print(df_trn.regulator.notnull().all())
print(df_trn.gene_id.notnull().all())

NameError: name 'df_trn' is not defined

## Load the data
You're now ready to load your IcaData object!

In [4]:
gene_table = pd.read_csv(path.join(data_dir,'gene_info.csv'))
gene_table

,locus_tag,gene_name,accession,old_locus_tag,start,end,strand,gene_product,COG,uniprot,operon
0,SGL_RS01370,SGL_RS01370,NC_005232.1,slr6001,243,3053,+,hybrid sensor histidine kinase/response regulator,Signal transduction mechanisms,NaN,Op97
1,SGL_RS00010,SGL_RS00010,NC_005229.1,sll5002,574,1017,-,hypothetical protein,No COG annotation,NaN,Op712
2,SGL_RS00635,SGL_RS00635,NC_005230.1,sll7002,706,1287,-,IS701 family transposase,"Replication, recombination and repair",NaN,Op2588
3,SGL_RS01880,SGL_RS01880,NC_000911.1,slr0612,811,1494,+,pseudouridine synthase,"Translation, ribosomal structure and biogenesis",NaN,Op465
4,SGL_RS00015,SGL_RS00015,NC_005229.1,sll5003,1136,1540,-,DUF5615 family PIN-like protein,No COG annotation,NaN,Op686
...,...,...,...,...,...,...,...,...,...,...,...
3737,SGL_RS18370,SGL_RS18370,NC_000911.1,slr0607,3569134,3569523,+,cyclic nucleotide-binding domain-containing pr...,Signal transduction mechanisms,NaN,Op766
3738,SGL_RS18375,hisIE,NC_000911.1,slr0608,3569672,3570319,+,bifunctional phosphoribosyl-AMP cyclohydrolase...,Nucleotide transport and metabolism,A0A6P1VM37_9SYNC,Op2513
3739,SGL_RS18380,SGL_RS18380,NC_000911.1,slr0609,3570424,3571575,+,GTP-binding protein,Function unknown,NaN,Op489
3740,SGL_RS18385,SGL_RS18385,NC_000911.1,slr0610,3571711,3572403,+,ABC transporter permease,Function unknown,NaN,Op108


In [5]:
#index is duplicated for uniprot records, so merge this cell values in one row
problematic_colum = 'uniprot'
duplicated_locus = set([ t for t in gene_table['locus_tag'] if gene_table['locus_tag'].tolist().count(t)>1])

if len(duplicated_locus)>0:
    replace_dict = {}
    for locus in duplicated_locus:
        group = gene_table.loc[gene_table['locus_tag']==locus]
        new_value = ';'.join(group[problematic_colum].tolist())
        for u_id in group[problematic_colum].tolist():
            replace_dict[u_id] = new_value

gene_table.replace({ problematic_colum : replace_dict},inplace=True)
gene_table.drop_duplicates(inplace=True)
gene_table = gene_table.set_index('locus_tag')

display(gene_table.head())
print(len(gene_table))

,gene_name,accession,old_locus_tag,start,end,strand,gene_product,COG,uniprot,operon
locus_tag,,,,,,,,,,
SGL_RS01370,SGL_RS01370,NC_005232.1,slr6001,243,3053,+,hybrid sensor histidine kinase/response regulator,Signal transduction mechanisms,NaN,Op97
SGL_RS00010,SGL_RS00010,NC_005229.1,sll5002,574,1017,-,hypothetical protein,No COG annotation,NaN,Op712
SGL_RS00635,SGL_RS00635,NC_005230.1,sll7002,706,1287,-,IS701 family transposase,"Replication, recombination and repair",NaN,Op2588
SGL_RS01880,SGL_RS01880,NC_000911.1,slr0612,811,1494,+,pseudouridine synthase,"Translation, ribosomal structure and biogenesis",NaN,Op465
SGL_RS00015,SGL_RS00015,NC_005229.1,sll5003,1136,1540,-,DUF5615 family PIN-like protein,No COG annotation,NaN,Op686


3690


In [18]:
ica_data = IcaData(M = path.join(data_dir,'M.csv'),
                   A = path.join(data_dir,'A.csv'),
                   X = path.join(data_dir,'log_tpm_norm.csv'),
                   gene_table = gene_table,
                   sample_table = path.join(data_dir,'final_metadata.tsv'),
#                  trn = path.join(data_dir,'TRN.csv'),
                   threshold_method='kmeans')

If you don't have a TRN (or have a very minimal TRN), use `threshold_method = 'kmeans'`

In [57]:
# ica_data = IcaData(M = path.join(data_dir,'M.csv'),
#                    A = path.join(data_dir,'A.csv'),
#                    X = path.join(data_dir,'log_tpm_norm.csv'),
#                    gene_table = path.join(data_dir,'gene_info.csv'),
#                    sample_table = path.join(data_dir,'metadata.tsv'),
#                    trn = path.join(data_dir,'TRN.csv'),
#                    threshold_method = 'kmeans')

# Regulatory iModulons
Use `compute_trn_enrichment` to automatically check for Regulatory iModulons. The more complete your TRN, the more regulatory iModulons you'll find.

In [19]:
ica_data.compute_trn_enrichment()

AttributeError: 'DataFrame' object has no attribute 'gene_id'

You can also search for AND/OR combinations of regulators using the `max_regs` argument.

Regulator enrichments can be directly saved to the `imodulon_table` using the `save` argument. This saves the enrichment with the lowest q-value to the table.

In [20]:
# First search for regulator enrichments with 2 regulators
ica_data.compute_trn_enrichment(max_regs=2,save=True)

# Next, search for regulator enrichments with just one regulator. This will supercede the 2 regulator enrichments.
ica_data.compute_trn_enrichment(max_regs=1,save=True)

AttributeError: 'DataFrame' object has no attribute 'gene_id'

The list of regulatory iModulons are shown below

In [21]:
regulatory_imodulons = ica_data.imodulon_table[ica_data.imodulon_table.regulator.notnull()]
print(len(ica_data.imodulon_table),'Total iModulons')
print(len(regulatory_imodulons),'Regulatory iModulons')
regulatory_imodulons

AttributeError: 'DataFrame' object has no attribute 'regulator'

You can rename iModulons in this jupyter notebook, or you can save the iModulon table as a CSV and edit it in Excel.

If two iModulons have the same regulator (e.g. 'Reg'), they will be named 'Reg-1' and 'Reg-2'

In [22]:
ica_data.rename_imodulons(regulatory_imodulons.regulator.to_dict())
ica_data.imodulon_table.head()

NameError: name 'regulatory_imodulons' is not defined

In [23]:
regulatory_imodulons = ica_data.imodulon_table[ica_data.imodulon_table.regulator.notnull()]

AttributeError: 'DataFrame' object has no attribute 'regulator'

# Functional iModulons

GO annotations and KEGG pathways/modules were generated in the 1_create_the_gene_table.ipynb notebook. Enrichments will be calculated in this notebook, and further curated in the 3_manual_iModulon_curation notebook.

## GO Enrichments

First load the Gene Ontology annotations

In [24]:
DF_GO = pd.read_csv(path.join(external_data,'GO_annotations_curated.csv'),index_col=0)
display(DF_GO.head())
print(len(DF_GO))

,gene_id,gene_name,gene_ontology
3,sll0537,sll0537,ammonium homeostasis
4,sll0537,sll0537,ammonium transmembrane transport
5,sll0601,sll0601,N-carbamoylputrescine amidase activity
6,sll0601,sll0601,putrescine biosynthetic process from arginine
7,slr1145,gltS,glutamate synthase (NADH) activity


2254


In [25]:
gene_table = gene_table.reset_index()
gene_table = gene_table.rename(columns={'index':'locus_tag'})
gene_table

,locus_tag,gene_name,accession,old_locus_tag,start,end,strand,gene_product,COG,uniprot,operon
0,SGL_RS01370,SGL_RS01370,NC_005232.1,slr6001,243,3053,+,hybrid sensor histidine kinase/response regulator,Signal transduction mechanisms,NaN,Op97
1,SGL_RS00010,SGL_RS00010,NC_005229.1,sll5002,574,1017,-,hypothetical protein,No COG annotation,NaN,Op712
2,SGL_RS00635,SGL_RS00635,NC_005230.1,sll7002,706,1287,-,IS701 family transposase,"Replication, recombination and repair",NaN,Op2588
3,SGL_RS01880,SGL_RS01880,NC_000911.1,slr0612,811,1494,+,pseudouridine synthase,"Translation, ribosomal structure and biogenesis",NaN,Op465
4,SGL_RS00015,SGL_RS00015,NC_005229.1,sll5003,1136,1540,-,DUF5615 family PIN-like protein,No COG annotation,NaN,Op686
...,...,...,...,...,...,...,...,...,...,...,...
3685,SGL_RS18370,SGL_RS18370,NC_000911.1,slr0607,3569134,3569523,+,cyclic nucleotide-binding domain-containing pr...,Signal transduction mechanisms,NaN,Op766
3686,SGL_RS18375,hisIE,NC_000911.1,slr0608,3569672,3570319,+,bifunctional phosphoribosyl-AMP cyclohydrolase...,Nucleotide transport and metabolism,A0A6P1VM37_9SYNC,Op2513
3687,SGL_RS18380,SGL_RS18380,NC_000911.1,slr0609,3570424,3571575,+,GTP-binding protein,Function unknown,NaN,Op489
3688,SGL_RS18385,SGL_RS18385,NC_000911.1,slr0610,3571711,3572403,+,ABC transporter permease,Function unknown,NaN,Op108


In [26]:
mapping_features = ['old_locus_tag','locus_tag']
mapping_df = gene_table.loc[gene_table.old_locus_tag.isin(DF_GO.gene_id.tolist())]
go_mapping_dict = mapping_df[mapping_features].set_index(mapping_features[0]).to_dict()[mapping_features[1]]
DF_GO.replace({'gene_id' : go_mapping_dict}, inplace=True)
DF_GO

,gene_id,gene_name,gene_ontology
3,SGL_RS16895,sll0537,ammonium homeostasis
4,SGL_RS16895,sll0537,ammonium transmembrane transport
5,SGL_RS14360,sll0601,N-carbamoylputrescine amidase activity
6,SGL_RS14360,sll0601,putrescine biosynthetic process from arginine
7,SGL_RS05620,gltS,glutamate synthase (NADH) activity
...,...,...,...
3351,SGL_RS02565,sll0225,cytoplasm
3354,SGL_RS02545,slr0245,transcription factor binding
3355,SGL_RS02545,slr0245,negative regulation of transcription by RNA po...
3356,SGL_RS02545,slr0245,histone deacetylase activity


In [30]:
DF_GO_enrich = ica_data.compute_annotation_enrichment(DF_GO,'gene_ontology')

In [31]:
display(DF_GO_enrich)
print(len(DF_GO_enrich))

,imodulon,gene_ontology,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size
0,1,structural constituent of ribosome,4.033821e-29,3.513458e-26,0.313433,0.583333,0.407767,21.0,36.0,67.0
1,1,cytosolic large ribosomal subunit,1.095099e-21,4.769155e-19,0.208955,0.736842,0.325581,14.0,19.0,67.0
2,1,cytosolic small ribosomal subunit,3.988856e-07,1.158098e-04,0.074627,0.500000,0.129870,5.0,10.0,67.0
3,1,translation,1.218767e-06,2.653865e-04,0.074627,0.416667,0.126582,5.0,12.0,67.0
4,1,RNA binding,5.577349e-05,9.715741e-03,0.044776,0.600000,0.083333,3.0,5.0,67.0
5,1,cytoplasmic translation,3.248506e-04,4.042070e-02,0.029851,1.000000,0.057971,2.0,2.0,67.0
6,1,large ribosomal subunit rRNA binding,3.248506e-04,4.042070e-02,0.029851,1.000000,0.057971,2.0,2.0,67.0
7,17,plasma membrane-derived thylakoid photosystem II,6.278898e-05,5.468920e-02,0.059701,0.307692,0.100000,4.0,16.0,67.0


8


In [76]:
set(ica_data.view_imodulon(1).index).intersection(set(DF_GO.loc[DF_GO['gene_ontology']=='structural constituent of ribosome'].gene_id.tolist()))

{'SGL_RS05715',
 'SGL_RS05720',
 'SGL_RS05730',
 'SGL_RS05765',
 'SGL_RS05770',
 'SGL_RS05780',
 'SGL_RS05785',
 'SGL_RS05790',
 'SGL_RS05800',
 'SGL_RS05805',
 'SGL_RS05815',
 'SGL_RS05820',
 'SGL_RS05825',
 'SGL_RS05830',
 'SGL_RS05835',
 'SGL_RS05845',
 'SGL_RS05850',
 'SGL_RS06195',
 'SGL_RS06200',
 'SGL_RS06210',
 'SGL_RS10920'}

In [69]:
set(ica_data.view_imodulon(17).index).intersection(set(DF_GO.loc[DF_GO['gene_ontology']=='plasma membrane-derived thylakoid photosystem II'].gene_id.tolist()))

{'SGL_RS08200', 'SGL_RS11935', 'SGL_RS14825', 'SGL_RS16825'}

In [83]:
imodulon_genes = set()
no_annotation = ['No COG annotation', 'Function unknown']
annotated_genes = DF_GO.loc[~DF_GO['gene_ontology'].isin(no_annotation)]
print('%s genes are annotated' % len(annotated_genes.gene_id.unique()))
for i in ica_data.imodulon_table.index:
    print( i , annotated_genes.loc[annotated_genes.gene_id.isin(ica_data.view_imodulon(i).index.tolist()), 'gene_ontology'].value_counts().max())
    imodulon_genes |= set(ica_data.view_imodulon(i).index.tolist())

print('%s / %s of imodulon genes are present in GO annotation table' % (len(imodulon_genes.intersection(set(annotated_genes.gene_id.tolist()))), len(imodulon_genes)))

856 genes are annotated
0 2
1 21
2 9
3 2
4 2
5 11
6 nan
7 8
8 1
9 8
10 nan
11 39
12 2
13 6
14 6
15 9
16 3
17 7
18 1
19 6
265 / 1309 of imodulon genes are present in GO annotation table


In [89]:
annotated_genes.loc[annotated_genes.gene_id.isin(ica_data.view_imodulon(17).index.tolist()), 'gene_ontology'].value_counts()

plasma membrane-derived thylakoid photosystem II                  7
protein binding                                                   7
plasma membrane-derived photosystem I                             2
ammonium homeostasis                                              1
translation                                                       1
cytoplasm                                                         1
glutamine biosynthetic process                                    1
membrane                                                          1
glutamate-ammonia ligase activity                                 1
water channel activity                                            1
plasma membrane                                                   1
mRNA binding                                                      1
cytosolic large ribosomal subunit                                 1
structural constituent of ribosome                                1
ammonium transmembrane transport                

## KEGG Enrichments

### Load KEGG mapping
The `kegg_mapping.csv` file contains KEGG orthologies, pathways, modules, and reactions. Only pathways and modules are relevant to iModulon characterization.

In [90]:
DF_KEGG = pd.read_csv(path.join(external_data,'kegg_mapping.csv'),index_col=0)
print(DF_KEGG.database.unique())
DF_KEGG.head()

['KEGG_orth' 'KEGG_pathway' 'KEGG_module' 'KEGG_reaction']


,gene_id,database,kegg_id
1,SGL_RS01885,KEGG_orth,-
2,SGL_RS01890,KEGG_orth,-
9,SGL_RS01925,KEGG_orth,-
15,SGL_RS01955,KEGG_orth,-
17,SGL_RS01985,KEGG_orth,-


In [91]:
kegg_pathways = DF_KEGG[DF_KEGG.database == 'KEGG_pathway']
kegg_modules = DF_KEGG[DF_KEGG.database == 'KEGG_module']

### Perform enrichment
Uses the `compute_annotation_enrichment` function

In [104]:
DF_pathway_enrich = ica_data.compute_annotation_enrichment(kegg_pathways,'kegg_id')
DF_module_enrich = ica_data.compute_annotation_enrichment(kegg_modules,'kegg_id')

In [105]:
DF_pathway_enrich.head()

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size
0,0,map04141,2.962003e-04,6.249826e-02,0.031250,1.000000,0.060606,2.0,2.0,64.0
1,1,map03010,1.563306e-33,3.298575e-31,0.388060,0.490566,0.433333,26.0,53.0,67.0
2,3,map02010,5.167328e-09,1.090306e-06,0.321429,0.107143,0.160714,9.0,84.0,28.0
3,9,map02010,6.517585e-06,1.375210e-03,0.212121,0.083333,0.119658,7.0,84.0,33.0
4,13,map00196,2.623274e-05,5.535109e-03,0.052083,0.333333,0.090090,5.0,15.0,96.0


In [106]:
DF_module_enrich.head()

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size
0,1,M00178,1.563306e-33,3.361107e-31,0.388060,0.490566,0.433333,26.0,53.0,67.0
1,1,M00179,1.114389e-31,1.197968e-29,0.313433,0.700000,0.432990,21.0,30.0,67.0
2,3,M00222,1.087344e-18,2.337790e-16,0.321429,0.818182,0.461538,9.0,11.0,28.0
3,4,M00316,4.350375e-08,9.353306e-06,0.214286,1.000000,0.352941,3.0,3.0,14.0
4,5,M00246,5.352880e-05,1.150869e-02,0.012618,1.000000,0.024922,4.0,4.0,317.0


### Convert KEGG IDs to human-readable names

In [107]:
for idx,key in tqdm(DF_pathway_enrich.kegg_id.items(),total=len(DF_pathway_enrich)):
    text = REST.kegg_find('pathway',key).read()
    try:
        name = re.search('\t(.*)\n',text).group(1)
        DF_pathway_enrich.loc[idx,'pathway_name'] = name
    except AttributeError:
        DF_pathway_enrich.loc[idx,'pathway_name'] = None
    
for idx,key in tqdm(DF_module_enrich.kegg_id.items(),total=len(DF_module_enrich)):
    text = REST.kegg_find('module',key).read()
    try:
        name = re.search('\t(.*)\n',text).group(1)
        DF_module_enrich.loc[idx,'module_name'] = name
    except AttributeError:
        DF_module_enrich.loc[idx,'module_name'] = None

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

In [108]:
DF_pathway_enrich

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size,pathway_name
0,0,map04141,2.962003e-04,6.249826e-02,0.031250,1.000000,0.060606,2.0,2.0,64.0,Protein processing in endoplasmic reticulum
1,1,map03010,1.563306e-33,3.298575e-31,0.388060,0.490566,0.433333,26.0,53.0,67.0,Ribosome
2,3,map02010,5.167328e-09,1.090306e-06,0.321429,0.107143,0.160714,9.0,84.0,28.0,ABC transporters
3,9,map02010,6.517585e-06,1.375210e-03,0.212121,0.083333,0.119658,7.0,84.0,33.0,ABC transporters
4,13,map00196,2.623274e-05,5.535109e-03,0.052083,0.333333,0.090090,5.0,15.0,96.0,Photosynthesis - antenna proteins
5,14,map00910,3.164213e-10,6.676489e-08,0.230769,0.400000,0.292683,6.0,15.0,26.0,Nitrogen metabolism
6,14,map02010,2.448962e-04,1.498222e-02,0.192308,0.059524,0.090909,5.0,84.0,26.0,ABC transporters
7,14,map04724,1.426302e-04,1.498222e-02,0.076923,0.666667,0.137931,2.0,3.0,26.0,Glutamatergic synapse
8,14,map04727,2.840232e-04,1.498222e-02,0.076923,0.500000,0.133333,2.0,4.0,26.0,GABAergic synapse
9,14,map04217,4.713197e-04,1.988969e-02,0.076923,0.400000,0.129032,2.0,5.0,26.0,Necroptosis


In [109]:
DF_module_enrich

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size,module_name
0,1,M00178,1.563306e-33,3.361107e-31,0.388060,0.490566,0.433333,26.0,53.0,67.0,NaN
1,1,M00179,1.114389e-31,1.197968e-29,0.313433,0.700000,0.432990,21.0,30.0,67.0,NaN
2,3,M00222,1.087344e-18,2.337790e-16,0.321429,0.818182,0.461538,9.0,11.0,28.0,NaN
3,4,M00316,4.350375e-08,9.353306e-06,0.214286,1.000000,0.352941,3.0,3.0,14.0,NaN
4,5,M00246,5.352880e-05,1.150869e-02,0.012618,1.000000,0.024922,4.0,4.0,317.0,NaN
5,5,M00245,6.285372e-04,6.756775e-02,0.009464,1.000000,0.018750,3.0,3.0,317.0,NaN
6,9,M00240,2.636184e-08,5.667795e-06,0.121212,0.800000,0.210526,4.0,5.0,33.0,NaN
7,9,M00190,6.441446e-06,6.924554e-04,0.090909,0.600000,0.157895,3.0,5.0,33.0,NaN
8,13,M00155,3.225298e-04,6.934391e-02,0.031250,0.500000,0.058824,3.0,6.0,96.0,"Cytochrome c oxidase, prokaryotes"
9,13,M00154,6.699769e-04,7.202251e-02,0.020833,1.000000,0.040816,2.0,2.0,96.0,Cytochrome c oxidase


In [111]:
module_dict = {'M00178' : 'Ribosome, bacteria',
               'M00179' : 'Ribosome, archaea',
               'M00222' : 'Phosphate transport system',
               'M00316' : 'Manganese transport system',
               'M00246' : 'Nickel transport system',
               'M00245' : 'Cobalt/nickel transport system',
               'M00240' : 'Iron complex transport system',
               'M00190' : 'Iron(III) transport system',
               'M00323' : 'Urea transport system',
               'M00438' : 'Nitrate/nitrite transport system',
              }

DF_module_enrich['module_name'] = [ module_dict[row['kegg_id']] if row['kegg_id'] in module_dict.keys() else row['module_name']
                                    for index, row in DF_module_enrich.iterrows() ]
DF_module_enrich

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size,module_name
0,1,M00178,1.563306e-33,3.361107e-31,0.388060,0.490566,0.433333,26.0,53.0,67.0,"Ribosome, bacteria"
1,1,M00179,1.114389e-31,1.197968e-29,0.313433,0.700000,0.432990,21.0,30.0,67.0,"Ribosome, archaea"
2,3,M00222,1.087344e-18,2.337790e-16,0.321429,0.818182,0.461538,9.0,11.0,28.0,Phosphate transport system
3,4,M00316,4.350375e-08,9.353306e-06,0.214286,1.000000,0.352941,3.0,3.0,14.0,Manganese transport system
4,5,M00246,5.352880e-05,1.150869e-02,0.012618,1.000000,0.024922,4.0,4.0,317.0,Nickel transport system
5,5,M00245,6.285372e-04,6.756775e-02,0.009464,1.000000,0.018750,3.0,3.0,317.0,Cobalt/nickel transport system
6,9,M00240,2.636184e-08,5.667795e-06,0.121212,0.800000,0.210526,4.0,5.0,33.0,Iron complex transport system
7,9,M00190,6.441446e-06,6.924554e-04,0.090909,0.600000,0.157895,3.0,5.0,33.0,Iron(III) transport system
8,13,M00155,3.225298e-04,6.934391e-02,0.031250,0.500000,0.058824,3.0,6.0,96.0,"Cytochrome c oxidase, prokaryotes"
9,13,M00154,6.699769e-04,7.202251e-02,0.020833,1.000000,0.040816,2.0,2.0,96.0,Cytochrome c oxidase


## SubtiWiki categories

In [ ]:
DF_subtiwiki = pd.read_csv(path.join(external_data,'subtiwiki_categories.csv'))
DF_subtiwiki.head()

,SubtiWiki,BSU_number,FuncId,FuncName1,FuncName2,FuncName3,FuncName4,FuncName5
0,dat,BSU_09670,SW 1.1,Cellular processes,cell envelope and cell division,cell wall synthesis,biosynthesis of peptidoglycan,NaN
1,ddl,BSU_04560,SW 1.1,Cellular processes,cell envelope and cell division,cell wall synthesis,biosynthesis of peptidoglycan,NaN
2,glmM,BSU_01770,SW 1.1,Cellular processes,cell envelope and cell division,cell wall synthesis,biosynthesis of peptidoglycan,NaN
3,glmS,BSU_01780,SW 1.1,Cellular processes,cell envelope and cell division,cell wall synthesis,biosynthesis of peptidoglycan,NaN
4,mraY,BSU_15190,SW 1.1,Cellular processes,cell envelope and cell division,cell wall synthesis,biosynthesis of peptidoglycan,NaN


In [ ]:
# Change the subtiwiki annotation format into a list of genes and categories
DF_subtiwiki = DF_subtiwiki.rename({'BSU_number':'gene_id'},axis=1)
DF_subtiwiki = DF_subtiwiki.melt(id_vars='gene_id',value_vars=['FuncName1','FuncName2','FuncName3','FuncName4','FuncName5'])
DF_subtiwiki = DF_subtiwiki[DF_subtiwiki.value.notnull() & DF_subtiwiki.gene_id.isin(ica_data.gene_names)]
DF_subtiwiki.head()

,gene_id,variable,value
0,BSU_09670,FuncName1,Cellular processes
1,BSU_04560,FuncName1,Cellular processes
2,BSU_01770,FuncName1,Cellular processes
3,BSU_01780,FuncName1,Cellular processes
4,BSU_15190,FuncName1,Cellular processes


In [ ]:
DF_subti_enrich = ica_data.compute_annotation_enrichment(DF_subtiwiki,'value')

## Save files

In [112]:
DF_GO_enrich['source'] = 'GO'
DF_pathway_enrich['source'] = 'KEGG pathways'
DF_module_enrich['source'] = 'KEGG modules'
# DF_subti_enrich['source'] = 'SubtiWiki'

DF_GO_enrich.rename({'gene_ontology':'annotation'},axis=1, inplace=True)
DF_pathway_enrich.rename({'kegg_id':'annotation'},axis=1, inplace=True)
DF_module_enrich.rename({'kegg_id':'annotation'},axis=1, inplace=True)
# DF_subti_enrich.rename({'value':'annotation'},axis=1, inplace=True)

DF_enrichments = pd.concat([DF_GO_enrich, DF_pathway_enrich, DF_module_enrich])
display(DF_enrichments)
DF_enrichments.to_csv(path.join(data_dir,'functional_enrichments.csv'))

,imodulon,annotation,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size,source,pathway_name,module_name
0,1,structural constituent of ribosome,4.033821e-29,3.513458e-26,0.313433,0.583333,0.407767,21.0,36.0,67.0,GO,NaN,NaN
1,1,cytosolic large ribosomal subunit,1.095099e-21,4.769155e-19,0.208955,0.736842,0.325581,14.0,19.0,67.0,GO,NaN,NaN
2,1,cytosolic small ribosomal subunit,3.988856e-07,1.158098e-04,0.074627,0.500000,0.129870,5.0,10.0,67.0,GO,NaN,NaN
3,1,translation,1.218767e-06,2.653865e-04,0.074627,0.416667,0.126582,5.0,12.0,67.0,GO,NaN,NaN
4,1,RNA binding,5.577349e-05,9.715741e-03,0.044776,0.600000,0.083333,3.0,5.0,67.0,GO,NaN,NaN
5,1,cytoplasmic translation,3.248506e-04,4.042070e-02,0.029851,1.000000,0.057971,2.0,2.0,67.0,GO,NaN,NaN
6,1,large ribosomal subunit rRNA binding,3.248506e-04,4.042070e-02,0.029851,1.000000,0.057971,2.0,2.0,67.0,GO,NaN,NaN
7,17,plasma membrane-derived thylakoid photosystem II,6.278898e-05,5.468920e-02,0.059701,0.307692,0.100000,4.0,16.0,67.0,GO,NaN,NaN
0,0,map04141,2.962003e-04,6.249826e-02,0.031250,1.000000,0.060606,2.0,2.0,64.0,KEGG pathways,Protein processing in endoplasmic reticulum,NaN
1,1,map03010,1.563306e-33,3.298575e-31,0.388060,0.490566,0.433333,26.0,53.0,67.0,KEGG pathways,Ribosome,NaN


# Check for single gene iModulons

Some iModulons are dominated by a single, high-coefficient gene. These iModulons may result from:
1. Overdecomposition of the dataset to identify noisy genes
1. Artificial knock-out of single genes
1. Regulons with only one target gene

No matter what causes these iModulons, it is important to be aware of them. The find_single_gene_imodulons function identifies iModulons that are likely dominated by a single gene.

The iModulons identified by ``find_single_gene_imodulons`` may contain more than one gene, since a threshold-agnostic method is used to identify these iModulons.

In [113]:
sg_imods = ica_data.find_single_gene_imodulons(save=True)
len(sg_imods)

0

In [120]:
for i,mod in enumerate(sg_imods):
    ica_data.rename_imodulons({mod:'SG_'+str(i+1)})

In [ ]:
ica_data.imodulon_table[ica_data.imodulon_table.single_gene == True]

AttributeError: 'DataFrame' object has no attribute 'single_gene'

In [ ]:
ica_data.view_imodulon('SG_1')

KeyError: 'SG_1'

# Save iModulon object

In [114]:
from pymodulon.util import explained_variance
from pymodulon.io import *

In [115]:
ica_data.imodulon_names

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [116]:
# Add iModulon sizes and explained variance
for im in ica_data.imodulon_names:
    ica_data.imodulon_table.loc[im,'imodulon_size'] = len(ica_data.view_imodulon(im))
    ica_data.imodulon_table.loc[im,'explained_variance'] = explained_variance(ica_data,imodulons=im)

In [117]:
ica_data.imodulon_table

,imodulon_size,explained_variance
0,64.0,0.051002
1,67.0,0.022770
2,174.0,0.003296
3,28.0,0.006637
4,14.0,0.005571
5,317.0,0.022298
6,4.0,0.033855
7,72.0,0.014793
8,18.0,0.024013
9,33.0,0.026245


This will save your iModulon table, your thresholds, and any other information stored in the ica_data object.

In [118]:
save_to_json(ica_data, path.join(data_dir,'bsu_raw.json.gz'))

If you prefer to view and edit your iModulon table in excel, save it as a CSV and reload the iModulon as before

In [119]:
ica_data.imodulon_table.to_csv(path.join(data_dir,'imodulon_table_raw.csv'))